# 导入模块

In [21]:
import os

os.chdir(r"D:\SPO4Portfolio")
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
from scipy.optimize import minimize

start_date = pd.to_datetime("2016-01-01")

In [22]:
import numpy as np
import pandas as pd


def calc_sharpe(nav: pd.Series, annualize_factor=252):
    """
    基于 daily NAV 计算 Sharpe ratio（无风险利率=0）
    """
    daily_ret = nav.pct_change().dropna()
    mean = daily_ret.mean()
    std = daily_ret.std()

    if std == 0:
        return np.nan

    return np.sqrt(annualize_factor) * mean / std


def calc_max_drawdown(nav: pd.Series):
    """
    最大回撤（Max Drawdown）
    """
    cum_max = nav.cummax()
    drawdown = nav / cum_max - 1.0
    return drawdown.min()

# 数据准备

In [23]:
# 所有 ETF
tickers = ["EEM", "EFA", "JPXN", "SPY", "XLK", "VTI", "AGG", "DBC"]
n_assets = len(tickers)
equal_weight = np.full(n_assets, 1 / n_assets)  # 每个资产的权重都是 1/N

# 读取 log return
return_df = pd.DataFrame()
for ticker in tickers:
    df = pd.read_csv(f"data/FeatureData/{ticker}.csv", parse_dates=["Date"])
    df["Date"] = pd.to_datetime(df["Date"]).dt.normalize()
    df = df.set_index("Date")["log_return"].rename(ticker)
    return_df = pd.concat([return_df, df], axis=1)

# 等权投资

In [24]:
import numpy as np
import pandas as pd

# ===== 1. ETF 列表 =====
tickers = ["EEM", "EFA", "JPXN", "SPY", "XLK", "VTI", "AGG", "DBC"]
n_assets = len(tickers)
equal_weight = np.full(n_assets, 1 / n_assets)

# ===== 2. 读取 log return（按天）=====
return_df = []

for ticker in tickers:
    df = pd.read_csv(f"data/FeatureData/{ticker}.csv", parse_dates=["Date"])
    df["Date"] = pd.to_datetime(df["Date"]).dt.normalize()
    df = df.set_index("Date")["log_return"].rename(ticker)
    return_df.append(df)

return_df = pd.concat(return_df, axis=1).sort_index()

# 可选：限制时间区间（和你 rolling 对齐）
return_df = return_df.loc["2016-01-01":"2024-12-31"]

# ===== 3. log return → arithmetic return =====
arith_return = np.expm1(return_df)  # r_t

# ===== 4. 组合每日收益 =====
portfolio_daily_return = arith_return @ equal_weight

# ===== 5. 计算每日 NAV =====
nav = (1 + portfolio_daily_return).cumprod()

# ===== 6. 整理输出 =====
nav_df = pd.DataFrame({"date": nav.index, "nav": nav.values})
ew_nav = nav_df["nav"]

ew_sharpe = calc_sharpe(ew_nav)
ew_mdd = calc_max_drawdown(ew_nav)

print(f"[Equal Weight]")
print(f"Sharpe Ratio: {ew_sharpe:.3f}")
print(f"Max Drawdown: {ew_mdd:.2%}")

# ===== 7. 保存 =====
nav_df.to_csv("outputs/Baselines/EqualWeight.csv", index=False)

[Equal Weight]
Sharpe Ratio: 0.750
Max Drawdown: -27.25%


# 最大化夏普比率

## 夏普比率优化函数

In [25]:
def max_sharpe_portfolio(mu, cov, risk_free=0.0):
    n = len(mu)

    def neg_sharpe(w):
        port_return = np.dot(w, mu)
        port_vol = np.sqrt(np.dot(w, np.dot(cov, w)))
        return -((port_return - risk_free) / port_vol)

    constraints = {"type": "eq", "fun": lambda w: np.sum(w) - 1}
    bounds = [(0.0, 1.0)] * n
    w0 = np.array([1.0 / n] * n)

    result = minimize(neg_sharpe, w0, bounds=bounds, constraints=constraints)
    return result.x  # 最优权重

## 月度回测代码

In [26]:
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta

# ===== 参数 =====
tickers = ["EEM", "EFA", "JPXN", "SPY", "XLK", "VTI", "AGG", "DBC"]
start_date = pd.Timestamp("2016-01-01")
months = 108  # 2016-01 ~ 2024-12

nav_records = []
current_nav = 1.0

for i in range(months):
    infer_start = start_date + relativedelta(months=i)
    infer_end = (infer_start + relativedelta(months=1)) - pd.Timedelta(days=1)
    train_start = infer_start - relativedelta(years=1)
    train_end = infer_start - pd.Timedelta(days=1)

    try:
        # ===== 1. 训练区间（日收益）=====
        train_data = return_df.loc[train_start:train_end, tickers]
        train_return = np.expm1(train_data)

        daily_mu = train_return.mean()
        daily_cov = train_return.cov()

        # 年化
        mu_annual = daily_mu * 252
        cov_annual = daily_cov * 252

        # ===== 2. Max Sharpe 权重 =====
        weights = max_sharpe_portfolio(mu_annual.values, cov_annual.values)

        # ===== 3. 月内每日收益 =====
        test_data = return_df.loc[infer_start:infer_end, tickers]
        test_return = np.expm1(test_data)

        for date, r in test_return.iterrows():
            daily_ret = np.dot(r.values, weights)
            current_nav *= 1 + daily_ret

            nav_records.append({"date": date, "nav": current_nav})

    except Exception as e:
        print(f"⚠️ {infer_start.strftime('%Y-%m')} 出错：{e}")
        continue
ms_nav = df_maxsharpe_nav.set_index("date")["nav"]

ms_sharpe = calc_sharpe(ms_nav)
ms_mdd = calc_max_drawdown(ms_nav)

print(f"[Markowitz Max Sharpe]")
print(f"Sharpe Ratio: {ms_sharpe:.3f}")
print(f"Max Drawdown: {ms_mdd:.2%}")

# ===== 4. 输出 =====
df_maxsharpe_nav = pd.DataFrame(nav_records)
df_maxsharpe_nav.to_csv("outputs/Baselines/MaxSharpe.csv", index=False)

d:\Anaconda3\envs\DLinear\lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
d:\Anaconda3\envs\DLinear\lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
d:\Anaconda3\envs\DLinear\lib\site-packages\pandas\core\frame.py:10866: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
d:\Anaconda3\envs\DLinear\lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
d:\Anaconda3\envs\DLinear\lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


[Markowitz Max Sharpe]
Sharpe Ratio: 0.553
Max Drawdown: -26.33%
